In [1]:
import os

print("CE: ", len(os.listdir("../input/mayo-clinic/First Experiment/train/CE")))
print("LAA: ", len(os.listdir("../input/mayo-clinic/First Experiment/train/LAA")))

CE:  81630
LAA:  43207


In [2]:
directory = "../input/mayo-clinic/First Experiment/train"
model_path = "./mayo_eff_net_mdl_wts.hdf5"

# https://tfhub.dev/google/collections/efficientnet_v2/1
IMG_SIZE = 300
BATCH_SIZE = 32
EPOCHS = 20

In [3]:
from IPython.display import clear_output

!pip install -q tensorflow==2.4.1

clear_output()

In [4]:
import tensorflow as tf

# train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
#    directory,
#    labels="inferred",
#    label_mode="categorical",
#    class_names=["CE", "LAA"],
#     color_mode="rgb",
#     batch_size=32,
#     image_size=(IMG_SIZE, IMG_SIZE),
#     shuffle=True,
#     seed=1,
#     validation_split=0.2,
#     subset="training",
#     interpolation="bilinear",
#     follow_links=False
# )

# val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
#     directory,
#     labels="inferred",
#     label_mode="categorical",
#     class_names=["CE", "LAA"],
#     color_mode="rgb",
#     batch_size=32,
#     image_size=(IMG_SIZE, IMG_SIZE),
#     shuffle=True,
#     seed=1,
#     validation_split=0.2,
#     subset="validation",
#     interpolation="bilinear",
#     follow_links=False
# )

Upsample data

In [5]:
def augment_images(image, label):
    image = tf.image.random_flip_left_right(image)
    return image, label

# result = train_dataset.filter(lambda image, label: tf.math.reduce_all(tf.math.equal(label, [0., 1.]))).map(augment_images)
# train_dataset = train_dataset.concatenate(result)

In [6]:
# from tensorflow.keras.applications.efficientnet_v2 import *
from tensorflow.keras.applications.efficientnet import EfficientNetB3
from tensorflow.keras import layers

def build_model():
    inputs = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3))
#     model = tf.keras.applications.efficientnet_v2.EfficientNetV2B3(include_top=False, input_tensor=inputs, weights="imagenet")
    model = EfficientNetB3(include_top=False, input_tensor=inputs, weights=None)
    x = layers.GlobalAveragePooling2D(name="avg_pool")(model.output)
    x = layers.BatchNormalization()(x)
    top_dropout_rate = 0.1
    x = layers.Dropout(top_dropout_rate, name="top_dropout")(x)
    outputs = layers.Dense(2, activation="sigmoid", name="pred")(x)
    model = tf.keras.Model(inputs, outputs, name="EfficientNet")
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-2)
    model.compile(optimizer=optimizer, loss="binary_crossentropy", metrics=["binary_accuracy"])
    return model

In [7]:
from tensorflow.keras.callbacks import ModelCheckpoint

model = build_model()
epochs = 25
model_checkpoint_callback = ModelCheckpoint(
    filepath=model_path,
    save_weights_only=True,
    monitor='val_loss',
    mode='min',
    save_best_only=True)

model.load_weights("../input/mayo-effnet-weights-v1/mayo_eff_net_mdl_wts.hdf5")
# model.fit(train_dataset, batch_size=BATCH_SIZE, epochs=EPOCHS, validation_data=val_dataset, verbose=1, callbacks=[model_checkpoint_callback], shuffle=False)

2022-10-06 21:37:03.480687: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-06 21:37:03.581037: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-06 21:37:03.581782: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-10-06 21:37:03.583327: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compil

In [8]:
# model.save("./mayo_eff_net_mdl_wts.hdf5")

In [9]:
import pandas as pd
from tqdm import tqdm
import numpy as np

test_directory = "../input/tiling-example/usable"
df = pd.DataFrame(columns=["patient_id", "CE", "LAA"])
for patient_id in tqdm(os.listdir(test_directory)):
#     print(test_directory + "/" + patient_id + "/")
    test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        test_directory + "/" + patient_id + "/",
        label_mode=None,
        class_names=None,
        color_mode="rgb",
        batch_size=32,
        image_size=(IMG_SIZE, IMG_SIZE),
        shuffle=False,
        seed=None,
        validation_split=None,
        subset=None,
        interpolation="bilinear",
        follow_links=False
    )
    preds = model.predict(test_dataset)
    df.loc[len(df.index)] = [patient_id, np.float64(preds.mean(axis=0)[0].round(6)), np.float64(preds.mean(axis=0)[1].round(6))]

  0%|          | 0/4 [00:00<?, ?it/s]2022-10-06 21:37:10.266690: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Found 476 files belonging to 1 classes.


2022-10-06 21:37:13.165768: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005
 25%|██▌       | 1/4 [00:11<00:33, 11.04s/it]

Found 1826 files belonging to 1 classes.


 50%|█████     | 2/4 [00:18<00:18,  9.22s/it]

Found 567 files belonging to 1 classes.


 75%|███████▌  | 3/4 [00:21<00:06,  6.20s/it]

Found 135 files belonging to 1 classes.


100%|██████████| 4/4 [00:22<00:00,  5.66s/it]


In [10]:
df = df.sort_values("patient_id")
df = df.reset_index(drop=True)
display(df)
df.to_csv("submission.csv", index=False)

test_csv  = pd.read_csv('../input/mayo-clinic-strip-ai/sample_submission.csv')
display(test_csv)
submission = test_csv
submission = submission.sort_values("patient_id")
submission["CE"] = df["CE"]
submission["LAA"] = df["LAA"]
# submission = submission.groupby("patient_id").mean()
display(submission)

submission.to_csv("submission.csv", index=False)

,patient_id,CE,LAA
0,006388,0.737600,0.265473
1,008e5c,0.739754,0.263429
2,00c058,0.698929,0.304127
3,01adc5,0.443714,0.559946


,patient_id,CE,LAA
0,006388,0.5,0.5
1,008e5c,0.5,0.5
2,00c058,0.5,0.5
3,01adc5,0.5,0.5


,patient_id,CE,LAA
0,006388,0.737600,0.265473
1,008e5c,0.739754,0.263429
2,00c058,0.698929,0.304127
3,01adc5,0.443714,0.559946


In [11]:
# print(df["patient_id"])
# print(df["CE"])
# print(df["LAA"])

In [12]:
# print(test_csv["patient_id"])
# print(test_csv["CE"])
# print(test_csv["LAA"])